In [122]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
%matplotlib inline
with open('transaction-data-adhoc-analysis.json','r') as f:
    data=json.load(f)
    df=pd.DataFrame(data)
df

,address,birthdate,mail,name,sex,username,transaction_items,transaction_value,transaction_date
0,USNS Wagner\nFPO AE 00917,1926/03/11,watsonbrenda@gmail.com,Jennifer Campbell,F,veronicawells,"Exotic Extras,Beef Chicharon,(x4)",5196,2022/01/12
1,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"Exotic Extras,Beef Chicharon,(x3);HealthyKid 3...",12056,2022/01/12
2,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Gummy Vitamins,(x3);HealthyKid 3...",5500,2022/01/08
3,USCGC Benson\nFPO AP 48794,2016/07/08,moorelisa@gmail.com,Melissa Kim,F,cgilbert,"HealthyKid 3+,Yummy Vegetables,(x1)",500,2022/01/10
4,"33911 Hansen Loaf\nWest Sandra, ND 54188",1949/09/27,michael78@yahoo.com,Kristen Cooper,F,qhernandez,"Candy City,Orange Beans,(x3);Candy City,Gummy ...",2697,2022/01/24
...,...,...,...,...,...,...,...,...,...
83031,"96081 Harrison Brooks\nJenniferbury, MA 02124",2019/10/26,christina55@hotmail.com,Tammy Byrd,F,kmcconnell,"Exotic Extras,Kimchi and Seaweed,(x2);HealthyK...",3588,2022/06/05
83032,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Yummy Vegetables,(x1);Exotic Ext...",2095,2022/06/12
83033,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"Candy City,Orange Beans,(x4);Candy City,Gummy ...",3396,2022/06/15
83034,428 Kristopher Curve Apt. 206\nMcculloughville...,1913/11/04,melvinschmidt@hotmail.com,Donald Andersen,M,llowe,"HealthyKid 3+,Nutrional Milk,(x2);Candy City,G...",4280,2022/06/09


In [123]:
dfx = df.copy()
dfx = dfx[["transaction_items", "transaction_value"]]
dfx["transaction_items"] = dfx["transaction_items"].str.split(";")
dfx = dfx[dfx["transaction_items"].apply(len) == 1]
dfx["transaction_items"] = dfx["transaction_items"].apply(lambda x: x[0][:-1].split(",(x"))
dfx["quantity"] = dfx["transaction_items"].apply(lambda x: int(x[1]))
dfx["transaction_items"] = dfx["transaction_items"].apply(lambda x: x[0].strip())
dfx.drop_duplicates(subset="transaction_items", inplace=True)
dfx["transaction_value"] /= dfx["quantity"]
dfx.set_index(["transaction_items"], inplace=True)
dfx.drop(columns=["quantity"], inplace=True)
dfx = dfx.squeeze()
dfx.sort_values()

transaction_items
Candy City,Gummy Worms               150.0
Candy City,Orange Beans              199.0
HealthyKid 3+,Yummy Vegetables       500.0
Exotic Extras,Kimchi and Seaweed     799.0
Exotic Extras,Beef Chicharon        1299.0
HealthyKid 3+,Gummy Vitamins        1500.0
HealthyKid 3+,Nutrional Milk        1990.0
Name: transaction_value, dtype: float64

In [124]:
dfy = df2.copy()
dfy["transaction_items"] = dfy["transaction_items"].str.split(";")
dfy = dfy.explode("transaction_items").reset_index(drop=True)
dfy["transaction_items"] = dfy["transaction_items"].apply(lambda x: x[:-1].split(',(x'))
dfy["quantity"] = dfy["transaction_items"].apply(lambda x: int(x[1]))
dfy["transaction_items"] = dfy["transaction_items"].apply(lambda x: x[0])
dfy["transaction_value"] = dfy["quantity"] * dfy["transaction_items"].apply(lambda x: dfx.loc[x])
dfy["transaction_month"] = pd.to_datetime(dfy["transaction_date"]).dt.month
dfy["transaction_sales"] = dfy["transaction_value"] * dfy["quantity"]
dfy

,name,transaction_items,transaction_value,transaction_date,quantity,transaction_month,transaction_sales
0,Jennifer Campbell,"Exotic Extras,Beef Chicharon",5196.0,2022-01-12,4,1,20784.0
1,Melissa Kim,"Exotic Extras,Beef Chicharon",3897.0,2022-01-12,3,1,11691.0
2,Melissa Kim,"HealthyKid 3+,Nutrional Milk",7960.0,2022-01-12,4,1,31840.0
3,Melissa Kim,"Candy City,Orange Beans",199.0,2022-01-12,1,1,199.0
4,Melissa Kim,"HealthyKid 3+,Gummy Vitamins",4500.0,2022-01-08,3,1,13500.0
...,...,...,...,...,...,...,...
166114,Donald Andersen,"HealthyKid 3+,Nutrional Milk",3980.0,2022-06-09,2,6,7960.0
166115,Donald Andersen,"Candy City,Gummy Worms",300.0,2022-06-09,2,6,600.0
166116,Stephanie Russell,"HealthyKid 3+,Nutrional Milk",1990.0,2022-06-03,1,6,1990.0
166117,Stephanie Russell,"Candy City,Gummy Worms",450.0,2022-06-03,3,6,1350.0


In [125]:
table1=pd.pivot_table(dfy, values=['quantity'], index=['transaction_items'], columns='transaction_month', aggfunc='sum', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
table1

quantity                                   
transaction_month                       1      2      3      4      5      6
transaction_items                                                           
Candy City,Gummy Worms               9559   9996   9986  10043   9801   9934
Candy City,Orange Beans              9774  10037   9611   9914   9964  10106
Exotic Extras,Beef Chicharon         9665  10001   9816   9890  10028   9902
Exotic Extras,Kimchi and Seaweed     9676   9949   9967   9921   9773  10104
HealthyKid 3+,Gummy Vitamins         9681   9980  10145   9842   9948   9980
HealthyKid 3+,Nutrional Milk         9727   9691   9876   9786   9881   9767
HealthyKid 3+,Yummy Vegetables       9959  10256   9896   9861   9735   9722

In [126]:
table2=pd.pivot_table(dfy, values=['transaction_sales'], index=['transaction_items'], columns='transaction_month', aggfunc='sum', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
table2

transaction_sales                          \
transaction_month                                1           2           3   
transaction_items                                                            
Candy City,Gummy Worms                   4318050.0   4472100.0   4498200.0   
Candy City,Orange Beans                  5906718.0   6035073.0   5661749.0   
Exotic Extras,Beef Chicharon            37765827.0  38924535.0  37912614.0   
Exotic Extras,Kimchi and Seaweed        23038366.0  23679963.0  23854145.0   
HealthyKid 3+,Gummy Vitamins            43816500.0  44727000.0  45658500.0   
HealthyKid 3+,Nutrional Milk            58257250.0  57743830.0  58868180.0   
HealthyKid 3+,Yummy Vegetables          14964500.0  15463000.0  14743000.0   

                                                                      
transaction_month                          4           5           6  
transaction_items                                                     
Candy City,Gummy Worms             4533450.0   4402650.0   4494600.0  
Candy City,Orange Beans            5928608.0   5971990.0   6048804.0  
Exotic Extras,Beef Chicharon      38258148.0  39310338.0  38684220.0  
Exotic Extras,Kimchi and Seaweed  23812597.0  23480213.0  24342334.0  
HealthyKid 3+,Gummy Vitamins      44307000.0  44772000.0  44808000.0  
HealthyKid 3+,Nutrional Milk      58064220.0  58448290.0  58878130.0  
HealthyKid 3+,Yummy Vegetables    14654500.0  14524500.0  14558000.0

In [127]:
df["transaction_month"] = pd.to_datetime(df["transaction_date"]).dt.month

In [130]:
customers = df[['name','transaction_month']]
customers = pd.pivot_table(customers, index=['transaction_month'], columns=['name'], aggfunc=any, fill_value=False)
customers

name,Aaron Beasley,Aaron Brewer,Aaron Brown,Aaron Coffey,Aaron Davis,Aaron Escobar,Aaron Flores,Aaron Flowers,Aaron Garrett,Aaron Holder,...,Zachary Richard,Zachary Salinas,Zachary Scott,Zachary Smith,Zachary Torres,Zachary Valentine,Zachary Ware,Zachary Williams,Zachary Wilson,Zachary York
transaction_month,,,,,,,,,,,,,,,,,,,,,
1,False,True,False,True,True,False,True,True,True,False,...,True,False,True,True,False,True,True,True,False,True
2,False,True,False,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,False,True
3,False,True,True,True,False,False,True,False,True,True,...,True,False,True,True,True,True,True,True,False,True
4,True,True,True,True,True,True,True,True,False,True,...,True,True,True,True,True,True,True,True,True,True
5,True,False,True,True,True,True,False,False,True,True,...,True,True,True,True,True,True,True,False,True,True
6,True,True,True,False,True,False,True,True,True,True,...,True,True,True,True,True,True,True,False,True,True


In [131]:
repeater = customers.apply(lambda x: [i != 1 and all([x[i-1],x[i]]) for i in x.index]).sum(axis=1)
inactive = customers.apply(lambda x: [i != 1 and any(x[:i]) and not x[i] for i in x.index]).sum(axis=1)
engaged = customers.apply(lambda x: [all(x[:i]) for i in x.index]).sum(axis=1)


activity_status = pd.DataFrame({'Repeaters': repeater, 'Inactives': inactive, 'Engaged': engaged})
activity_status

,Repeaters,Inactives,Engaged
transaction_month,,,
1,0,0,6588
2,5172,1416,5172
3,5216,1747,4126
4,5154,1909,3289
5,5110,1917,2667
6,5193,1835,2190
